In [282]:
!pip install fuzzywuzzy

In [1]:
import json
import pandas as pd
from itertools import chain
import re
from fuzzywuzzy import fuzz, process

/anaconda3/envs/py36_ml/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
data = []
with open('found_artists_filtered.ndjson', 'rb') as f:
    for l in f.readlines():
        data.append(json.loads(l))
data = list(chain(data))
data = [[d[0], d[1], d[3]] for d in data]

In [3]:
txt = '1982  & ndash; 1985'
txt = '1982  & ndash; present'


r1 = re.findall(r"\d{4}",txt)
r1 = re.findall(r"(\d{4}|present)",txt)
print(r1)

['1982', 'present']


In [4]:
data[1]

['K (singer)',
 {'name': 'K',
  'background': 'solo_singer',
  'birth_name': 'Kang Yoon-sung',
  'origin': 'South Korea',
  'Instruments': 'Singing, Piano',
  'genre': 'J-pop R & B',
  'occupation': 'singer',
  'years_active': '2004–2010; 2013-',
  'label': 'Sony Music Entertainment, Studioseven Recordings',
  'website': 'http://www.k-official.com/'},
 {'name': [],
  'background': [],
  'birth_name': [],
  'alias': [],
  'birth_date': [],
  'death_date': [],
  'origin': [],
  'Instruments': [],
  'genre': ['J-pop', 'Rhythm and blues'],
  'occupation': [],
  'years_active': [],
  'label': ['Sony Music Entertainment', 'Studioseven Recordings'],
  'Related_acts': [],
  'website': []}]

In [10]:
source = []
for d in data:
    tmp = {}
    tmp['name'] = str(d[0])
    tmp['background'] = str(d[2].get('background'))
    tmp['years_active'] = d[2].get('years_active')
    tmp['genre'] = d[2].get('genre')
    tmp['origin'] = d[2].get('origin')
    tmp['associated_acts'] = d[2].get('associated_acts')
    
    #if tmp['name']:
    #    try:
    #        tmp['name'] = str(tmp['name']).replace('"', '´')
    #    except:
    #        print(tmp['name'])
    #        continue
                      
    if not d[2].get('background') and d[1].get('background'):
        tmp['background'] = str(d[1]['background'])
    backgrounds = ['group_or_band', 'solo_singer', 'non_vocal_instrumentalist', 'non_performing_personnel', 'temporary', 'None']
    
    if not tmp['background']:
        background = 'None'
    else:
        ratios = process.extractBests(tmp['background'], backgrounds)
        ratios = sorted([r[1] for r in ratios])
        if ratios[-1] == ratios[-2]:
            background = 'None'
        else:
            background = process.extractOne(tmp['background'], backgrounds)[0]
    tmp['background'] = background
    if not d[2].get('years_active') and d[1].get('years_active'):
        tmp['years_active'] = re.findall(r"(\d{4}|present)",d[1]['years_active'])
    elif not tmp['years_active']:
        tmp['years_active'] = []
    
    if len(tmp['years_active']) > 2:
        tmp['years_active'] = [tmp['years_active'][0], tmp['years_active'][-1]]
    elif len(tmp['years_active']) == 0:
        tmp['years_active'] = [0, 0]
    elif len(tmp['years_active']) == 1:
        if tmp['years_active'][0] == 'present':
            tmp['years_active'] = [0,'present']
        else:
            tmp['years_active'] = [tmp['years_active'][0],'present']
    for i,year in enumerate(tmp['years_active']):
        if year == 'present':
            tmp['years_active'][i] = 2021
        try:
            tmp['years_active'][i] = int(tmp['years_active'][i])
        except:
            tmp['years_active'][i] = 0
        
    if not d[2].get('associated_acts'):
        tmp['associated_acts'] = d[2].get('Associated_acts')
    source.append(tmp)

In [11]:
nodes = pd.DataFrame([d for d in source])

In [12]:
nodes.name = nodes.name.fillna('')
nodes.name = nodes.name.astype(str)

In [13]:
ratios = process.extractBests('solo_singer  < !-- mandatory field -- >      ', backgrounds)
ratios = sorted([r[1] for r in ratios])
if ratios[-1] == ratios[-2]:
    background = 'None'
else:
    background = process.extractOne('**Mandatory** Use one: solo_singer, non_vocal_instrumentalist, non_performing_personnel, or temporary ', backgrounds)[0]
background

'non_vocal_instrumentalist'

In [14]:
nodes.head(12)


,name,background,years_active,genre,origin,associated_acts
0,Rose Hill Drive,group_or_band,"[2003, 2021]","[Hard rock, Blues rock]","[Boulder, Colorado, Colorado]",None
1,K (singer),solo_singer,"[2004, 2013]","[J-pop, Rhythm and blues]",[],None
2,Zos Kia,group_or_band,"[1982, 1985]",[Experimental music],"[London, England]",None
3,SITD,group_or_band,"[1996, 2021]","[Industrial music, Electronic body music]","[Ruhr Area, Germany]",None
4,CoH (musician),non_vocal_instrumentalist,"[0, 0]","[Glitch music, Industrial music, Intelligent d...",[],"[Soisong, Richard Chartier, Nice Box]"
5,Jorge Cumbo,non_vocal_instrumentalist,"[0, 0]",[Andean music],None,"[Urubamba (band), Paul Simon]"
6,The Rave-Ups,group_or_band,"[0, 0]",[alternative rock],"[Pittsburgh, Pennsylvania]",None
7,Martha Redbone,solo_singer,"[0, 0]","[rhythm and blues, soul music]",[],None
8,Terje Isungset,temporary,"[0, 0]","[Jazz, Nordic folk music]",[],None
9,Viva Hate (band),group_or_band,"[2004, 2021]","[Psychobilly, hard rock, punk rock]","[California, United States]","[Tiger Army, AFI (band), The Force (band), Fea..."


In [15]:
nodes.background.astype(str).value_counts()[:20]


solo_singer                  27967
group_or_band                21283
non_vocal_instrumentalist     9984
non_performing_personnel      3648
temporary                     1198
None                           742
Name: background, dtype: int64

In [16]:
nodes.shape

(64822, 6)

In [17]:
from neo4j import GraphDatabase

In [18]:
uri = "neo4j://localhost:7687"
neo_diver = GraphDatabase.driver(uri, auth=("neo4j", "musikk"))

In [19]:
def create_artist(tx, artist):
    # print('create stock ', aksje)
    name = artist[1]
    background = artist[2]
    years_active_start = artist[3][0]
    years_active_end = artist[3][1]
    if years_active_end == 'present':
        years_active_end = """'present'"""
    quary = """Merge (u:Artist {name:"%s"})
                set u.background="%s", u.years_active_start=%s, u.years_active_end=%s"""
    tx.run(quary%(name, background, years_active_start, years_active_end))
    
def create_associated_acts_relation(tx, artist):
    # print('create stock ', aksje)
    name = artist[1]
    acts = artist[-1]
    if acts:
        for act in acts:
            quary = """Match (u:Artist {name:"%s"}) 
                        Merge (a:Artist {name:"%s"}) 
                        Merge (a)<-[:ASSOCIATED]-(u)
                        """
            tx.run(quary%(name, act))

In [20]:
node_error_index = []
with neo_diver.session() as session:
    for r in nodes.itertuples():
        try:
            session.write_transaction(create_artist, r)
        except:
            node_error_index.append(r[0])

In [22]:
relationship_error_index = []
with neo_diver.session() as session:
    for r in nodes.itertuples():
        try:
            session.write_transaction(create_associated_acts_relation, r)
        except:
            relationship_error_index.append(r[0])
        

In [23]:
print('node errors',len(node_error_index))
print('relationship errors',len(relationship_error_index))


node errors 133
relationship errors 194
